In [ ]:
# Pfad-Fix: Notebook liegt in notebooks/c_chatgpt_integration/
import sys, pathlib, ipywidgets as w, pandas as pd
from IPython.display import display, Markdown

PROJECT_ROOT = pathlib.Path.cwd().parents[1]             # …/vocdata
sys.path.insert(0, str(PROJECT_ROOT / "scripts"))         # scripts importierbar

from chatgpt_config.search import search                  # Logik
from chatgpt_config.summarize_pdf_files import summarize  # Kurz-Summary


In [ ]:
# B) UI-Elemente anlegen
# Eingabe- und Ausgabewidgets
txt_query   = w.Text(placeholder="Suchbegriff …", description="🔍")
btn_search  = w.Button(description="Suchen")
tbl_out     = w.Output()
ddl_docs    = w.Dropdown(description="Dokument:")
sum_out     = w.Output(layout={'border': '1px solid #ddd', 'padding': '6px'})

# Anzeige
display(txt_query, btn_search, tbl_out, ddl_docs, sum_out)


In [ ]:
# c) Callback-Funktionen
def run_search(_):
    # 1) Suche ausführen
    df = search(txt_query.value.strip(), 20)
    if df.empty:
        ddl_docs.options = []
        tbl_out.clear_output()
        with tbl_out:
            print("Keine Treffer.")
        return
    
    # 2) Tabelle zeigen
    tbl_out.clear_output()
    with tbl_out:
        display(df[["doc_id", "filename", "snippet"]])
    
    # 3) Dropdown füllen
    ddl_docs.options = [(f"{r.filename}", r.doc_id) for r in df.itertuples()]
    ddl_docs.value = None   # zurücksetzen

def show_summary(change):
    doc_id = change["new"]
    if doc_id:
        text = summarize(doc_id)
        sum_out.clear_output()
        with sum_out:
            display(Markdown(f"**Zusammenfassung (ID {doc_id})**"))
            display(Markdown(text))

# Events binden
btn_search.on_click(run_search)
ddl_docs.observe(show_summary, names="value")
